# Pip install all needed packages

In [1]:
!pip install gym

Defaulting to user installation because normal site-packages is not writeable


# Imports

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np

# Global variables

In [26]:
depth = 10
rows = 3
cols = 3

# Functions

In [50]:
# find coords of actions with algorithm
def swap(state, actions):
    if type(actions[0]) != tuple:
        actions = [actions]
    for action in actions:
        pos0, pos1 = action
        
        row0 = int((pos0-(pos0%rows))/cols)
        col0 = int(pos0 - row0*3)    
        row1 = int((pos1-(pos1%rows))/cols)
        col1 = int(pos1 - row1*3)
        
        for i in range(len(state)):
            state[i][row0][col0], state[i][row1][col1] = state[i][row1][col1], state[i][row0][col0]


In [35]:
state = np.arange(9).reshape((3,3))
actions = (2, 5)

swap(state, actions)

2
5


In [3]:
# swap is given actions which is a tuple of actions or a action, where every action is a tuple with the values
# of two qubits (x, y) whos values should be swaped. x and y are ints between 0 and 8 corresponding to 
# the following qubit notation:
#         [[0, 1, 2],
#          [3, 4, 5],
#          [6, 7, 8]]

# ex. of a tuple of actions: ((0, 3), (4, 5), (7, 8))

# ex. of a single action: (0, 1)

# in the case of a single action we make a list out of it so it's iterable to minimize code

def swap(state, actions):
    if type(actions[0]) != tuple:
        actions = [actions]
    for action in actions:
        if action == (0, 1) or action == (1, 0): 
            row0, col0 = 0, 0
            row1, col1 = 0, 1
        elif action == (1, 2) or action == (2, 1):  
            row0, col0 = 0, 2
            row1, col1 = 0, 1
        elif action == (0, 3) or action == (3, 0): 
            row0, col0 = 1, 0
            row1, col1 = 0, 0
        elif action == (4, 1) or action == (1, 4): 
            row0, col0 = 1, 1
            row1, col1 = 0, 1
        elif action == (2, 5) or action == (5, 2): 
            row0, col0 = 1, 2
            row1, col1 = 0, 2
        elif action == (3, 4) or action == (4, 3): 
            row0, col0 = 1, 1
            row1, col1 = 1, 0
        elif action == (4, 5) or action == (5, 4): 
            row0, col0 = 1, 2
            row1, col1 = 1, 1
        elif action == (3, 6) or action == (6, 3): 
            row0, col0 = 2, 0
            row1, col1 = 1, 0
        elif action == (4, 7) or action == (7, 4): 
            row0, col0 = 2, 1
            row1, col1 = 1, 1
        elif action == (5, 8) or action == (8, 5): 
            row0, col0 = 2, 2
            row1, col1 = 1, 2
        elif action == (7, 6) or action == (6, 7): 
            row0, col0 = 2, 1
            row1, col1 = 2, 0
        elif action == (7, 8) or action == (8, 7): 
            row0, col0 = 2, 2
            row1, col1 = 2, 1
        else:
            return

        #swap for all future states
        for i in range(len(state)):
            state[i][row0][col0], state[i][row1][col1] = state[i][row1][col1], state[i][row0][col0]

In [37]:
# getNeighbors returns a list of the qubit notations of all neighbors to a specific qubit. 
# I.e. qubits above, below, right and left of the specific qubit.

def getNeighbors(state, row_number, column_number):
    a = [[state[i][j] if  i >= 0 and i < len(state) and j >= 0 and j < len(state[0]) else -1
                    for j in range(column_number-1, column_number+2)]
                        for i in range(row_number-1, row_number+2)]
    return [a[0][1], a[1][0], a[1][2], a[2][1]]

In [38]:
#                         [[1,0,0], [[1,0,0],   [[1,0,0],         [[1,0,0],  
# Takes in a state like [  [1,0,2],  [1,0,2], ,  [1,0,2], , ... ,  [1,0,2], ] and checks if all the pairs of 
#                          [2,0,0]]  [2,0,0]]    [2,0,0]]          [2,0,0]] 
# numbers in the first slice are neighbors and if so returns True else returns False

def isExecutableState(state):
    for row in range(len(state[0])):
        for col in range(len(state[0][0])):
            if state[0][row][col] > 0:
                if not state[0][row][col] in getNeighbors(state[0], row, col):
                    return False

    return True

In [39]:
# We use this once to get all the different swap combinations. I.e. all acceptable combinations of one to four
# swaps. This are the different actions we cound make in one timestep.

def getPossibleActions(depth=4):
    state = np.arange(9).reshape((3,3))
    
    possibleActions = getPossibleActionsSub(state, [], depth)
    
    possibleActions = set(map(lambda x: tuple(sorted(x)), possibleActions ))
    
    possibleActions = list(possibleActions)
    possibleActions.append((0, 0))
    
    return possibleActions
    
def getPossibleActionsSub(state, used, depth):
    if depth == 0:
        return np.asarray([])
    
    possibleActions = []
    
    for i in range(len(state)):
        for j in range(len(state[0])):
            
            usedtmp = used.copy()
            
            if not state[i][j] in usedtmp:
                neighbors = getNeighbors(state, i, j)
                for neighbor in neighbors:
                    if neighbor >= 0 and  not (neighbor, state[i][j]) in possibleActions and not neighbor in usedtmp:
                        possibleActions.append((state[i][j], neighbor))
                        usedtmp.append(state[i][j])
                        usedtmp.append(neighbor)
 
                        for action in getPossibleActionsSub(state, usedtmp, depth-1):
                            if type(action) == tuple:
                                possibleActions.append([(state[i][j], neighbor), action])
                            elif type(action) == list:
                                action.append((state[i][j], neighbor))
                                possibleActions.append(action)
                                
        
    return possibleActions

In [40]:
def makeStateSlice():
    random = np.random.choice([3,5,7,9])
    stateSlice = np.ceil(np.arange(random)/2)
    stateSlice = np.append(stateSlice, np.zeros(9-random)).reshape((3,3))
    np.random.shuffle(stateSlice)
    return stateSlice

In [41]:
def makeState():
    state = np.zeros((10,3,3))
    for i in range(len(state)):
        state[i] = makeStateSlice()
    return state

# Tests of the functions

In [51]:
state = np.zeros((10,3,3))
for i in range(len(state)):
    state[i] = np.arange(9).reshape((3,3))

#print(state)

possibleActions = getPossibleActions()
print(len(possibleActions))
#print(possibleActions)

a = possibleActions[0]

print(a)

statetmp = np.arange(9).reshape((3,3))

swap(state, a)

print(state[0])

#for i in range(len(possibleActions)):
#output = set(map(lambda x: tuple(sorted(x)),possibleActions))

#print(len(output))

131
((2, 5), (7, 8))
[[0. 1. 5.]
 [3. 4. 2.]
 [6. 8. 7.]]


testing of the functions isExecutableState and swap

In [43]:
arr = np.zeros((10,3,3), dtype=int)
arr[0][0][1] = 1
arr[0][0][2] = 2
arr[0][1][1] = 1
arr[0][1][2] = 2
arr[1] = arr[0]

print(arr[0])
print(arr[1])
print(isExecutableState(arr))

swap(arr, (1, 2))
print(arr[0])
print(arr[1])
print(isExecutableState(arr))

swap(arr, (0, 0))
print(arr[0])
print(arr[1])
print(isExecutableState(arr))

[[0 1 2]
 [0 1 2]
 [0 0 0]]
[[0 1 2]
 [0 1 2]
 [0 0 0]]
True


ValueError: not enough values to unpack (expected 2, got 1)

Test function makeState

In [44]:
print(makeState())

[[[0. 1. 1.]
  [2. 2. 3.]
  [3. 0. 0.]]

 [[2. 2. 0.]
  [0. 1. 1.]
  [0. 0. 0.]]

 [[2. 2. 3.]
  [3. 0. 0.]
  [0. 1. 1.]]

 [[2. 2. 3.]
  [3. 4. 4.]
  [0. 1. 1.]]

 [[0. 1. 1.]
  [2. 2. 0.]
  [0. 0. 0.]]

 [[0. 1. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 1. 1.]
  [0. 0. 0.]
  [2. 2. 0.]]

 [[0. 1. 1.]
  [3. 0. 0.]
  [2. 2. 3.]]

 [[0. 1. 1.]
  [2. 2. 3.]
  [3. 4. 4.]]

 [[0. 1. 1.]
  [3. 0. 0.]
  [2. 2. 3.]]]


# Enviotment definition and sub functions

In [45]:
#Our enviorment
class Kvant(Env):
    def __init__(self):
        #array of possible actions
        self.possibleActions = getPossibleActions()
        #self.possibleActions = getPossibleActions(1) #this for only 1 swap at a time
        
        #Number of actions we can take
        self.action_space = Discrete(len(self.possibleActions))
        
        #
        self.observation_space = Box(low=np.zeros((10,3,3), dtype=int), high=np.zeros((10,3,3), dtype=int)+4)
        
        #The start state
        self.state = makeState()
        
        #max amount of swaps between layers
        self.maxSwaps = 5
        
        #max amount of layers per episode
        self.maxLayers = 10
        
    def step(self, action):
        
        actions = self.possibleActions[action]

        swap(self.state, actions)
        
        self.maxSwaps -= 1
        
        if isExecutableState(self.state):
            # if the current slice is Exicutable then give the reward 5
            reward = 5
            
            # reset the maxSwaps to 5
            self.maxSwaps = 5
            
            # remove the exicutable slice and add a new random slice at the tail
            self.state = np.roll(self.state, -1, axis = 0)
            self.state[9] = makeStateSlice()
            
            self.maxLayers -= 1
            
            # we are not done except if this was the last layer we can work on this episode
            if self.maxLayers <= 0:
                done = True
            else:
                done = False
            
        elif self.maxSwaps <= 0:
            done = True
            reward = -1
        
        else:
            done = False
            reward = -1
            
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        pass
    
    def reset(self):
        self.state = makeState()
        self.maxSwaps = 5
        self.maxLayers = 10
        return self.state

# Testing the enviorment

In [46]:
env = Kvant()

/home/surface/.local/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


## test just random actions

In [47]:
episodes = 1000
scores = []
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    scores.append(score)

TypeError: unsupported operand type(s) for %: 'tuple' and 'int'

In [48]:
for i in range(min(scores), max(scores)+1):
    if scores.count(i) != 0:
        print('Number of', i, 'is', scores.count(i))

ValueError: min() arg is an empty sequence